# Customising Banana for Your Analysis

Now you understand how to write Arcana Analysis classes and have an idea of what is already implemented in Banana you can customise the analysis implemented in it to your needs.

**Note:** Before doing any customisations, have a good look through the parameters of the class to see whether there is already a switch to do what you want.

There are a couple of ways you might want to extend an existing class:

* Adding/overwriting a new parameter or parameter option
* Add new derivatives and pipeline constructors
* Overwrite/modify an existing pipeline constructor

For this notebook we will use the `BasicBrainAnalysis` class again instead of a "real" class from Banana as the analysis in those classes typically takes too long for the time we have available for this workshop. However, the concepts are the same.

## Adding/Overwriting Parameters

Adding a new parameter to a class is pretty straightforward, simply extend the class and put your new parameter in `add_param_specs`.

In [ ]:
from nipype import config
config.enable_debug_mode()  # This is necessary due to a bug in one of the interfaces
from arcana import Analysis, AnalysisMetaClass, ParamSpec, SwitchSpec
from example.analysis import BasicBrainAnalysis


class MyExtendendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_param_specs = [
        ParamSpec('bet_frac', 0.1, desc="The fractional intensity threshold for BET")
    ]
    
print(MyExtendendedBasicBrainAnalysis.static_menu())

Alternatively if you just want to change the default value for an existing parameter you can override it

In [ ]:
from arcana import Analysis, AnalysisMetaClass, ParamSpec, SwitchSpec
from example.analysis import BasicBrainAnalysis


class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_param_specs = [
        ParamSpec('smoothing_fwhm', 2)
    ]
    
print(MyExtendedBasicBrainAnalysis.static_menu())

Notice how we have lost the description of the parameter because we didn't provide it in the overwritten version. Instead of having to type the same thing again we can generate a new version of the original `ParamSpec` with a new default value by accessing the original from BasicBrainAnalysis and using the `with_new_default` method

In [ ]:
class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_param_specs = [
        BasicBrainAnalysis.param_spec('smoothing_fwhm').with_new_default(2.0)
    ]
    
print(MyExtendedBasicBrainAnalysis.static_menu())

## Adding New Data-Specs

Adding new data-specs is the same as adding new parameters just append them to the `add_data_specs` list of your extended class

In [ ]:
from arcana import OutputFilesetSpec
from banana.file_format import nifti_gz_format


class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_data_specs = [
        OutputFilesetSpec('skull_mask', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull mask extracted from magnitude image"),
    ]
    
print(MyExtendedBasicBrainAnalysis.static_menu())

## Modifying and Overwriting Pipeline Constructor Methods

Of course if you add new parameters and/or data specs you will need to add or modify the pipelines that use/generate them. Adding a new method is straightforward, simply define it in your extended class. Likewise overriding a method you just need to name your pipeline constructor method as it is in the base class. However, in most cases you will just want to modify the pipeline instead, in which case we use the built-in `super`.

In [ ]:
from nipype.interfaces import fsl
fsl.BET.help()

In [ ]:
from arcana import OutputFilesetSpec
from banana.file_format import nifti_gz_format


class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_data_specs = [
        OutputFilesetSpec('skull_mask', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull mask extracted from magnitude image"),
    ]
    
    def brain_extraction_pipeline(self, **name_maps):
        pipeline = super().brain_extraction_pipeline(**name_maps)
        
        bet = pipeline.node('bet')

        # Set the input of the BET node so that it outputs a Skull mask
        bet.inputs.surfaces = True
        
        pipeline.connect_output('skull_mask', bet, 'skull_mask_file', nifti_gz_format)
        
        return pipeline

## Adding Default Initialisations and Output Methods

If your extension is specific to a particular study, it can make things more convenient to add a 'default' classmethod that instantiates the Analysis class with links to where the study data is stored.

Likewise, you can add methods for specific publication outputs (e.g. figures) to your class. This makes it easy to try to replicate your results on other datasets

In [ ]:
import os.path as op
from nilearn import plotting, image
from arcana import Dataset, FilesetFilter, SingleProc
from arcana import OutputFilesetSpec
from banana.file_format import nifti_gz_format


class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    add_data_specs = [
        OutputFilesetSpec('skull_mask', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull mask extracted from magnitude image"),
    ]
    
    def brain_extraction_pipeline(self, **name_maps):
        pipeline = super().brain_extraction_pipeline(**name_maps)
        
        bet = pipeline.node('bet')

        # Set the input of the BET node so that it outputs a Skull mask
        bet.inputs.surfaces = True
        
        pipeline.connect_output('skull_mask', bet, 'skull_mask_file', nifti_gz_format)
        
        return pipeline

    @classmethod
    def default(cls, name='my_analysis'):
         return cls(
            name,
            dataset=Dataset('output/sample-datasets/depth1', depth=1),
            processor=SingleProc('work'),
            inputs=[
                FilesetFilter('magnitude', '.*T1w$', is_regex=True)])

    def plot_slices(self, spec_name, title, subject_id='sub1', visit_id='VISIT'):
        plotting.plot_anat(
            image.load_img(self.data(spec_name).item(
                subject_id=subject_id, visit_id=visit_id).path),
            title=title, display_mode='z', dim=-1,
            cut_coords=[-20, -10, 0, 10, 20, 30])


We can then easily initalise the analysis, derive the data and plot the results by

In [ ]:
analysis = MyExtendedBasicBrainAnalysis.default()
analysis.derive('skull_mask')

In [ ]:
analysis.plot_slices('skull_mask', 'Skull Mask')

## Exercise 1

Modify the `smooth_mask_pipeline` so that it uses the 'sigma' parameter instead of the 'fwhm' parameter to define the smoothing kernel and then plot the results of `smooth_masked`. Note you to unset the 'fwhm' parameter you will need to use 'Undefined from `from traits.trait_base import Undefined`.

Try extending from the extended class above (i.e. doubly extended) to reimplement `smooth_mask_pipeline`.

NB: As you are changing the parameters used from the previously run analysis, you will need to either specify a new name for your analysis (e.g. 'my_second_analysis') or set the `reprocess` of the processor to `True` (e.g. `analysis.processor.reprocess = True`

In [ ]:
## Write your solution here

In [ ]:
from traits.trait_base import Undefined


class MyExtendedExtendedBasicBrainAnalysis(MyExtendedBasicBrainAnalysis, metaclass=AnalysisMetaClass):
    
    def smooth_mask_pipeline(self, **name_maps):
        pipeline = super().smooth_mask_pipeline(**name_maps)
        
        smooth = pipeline.node('smooth')

        # Set the input of the BET node so that it outputs a Skull mask
        smooth.inputs.fwhm = Undefined
        smooth.inputs.sigma = 2.0
        
        return pipeline

In [ ]:
analysis = MyExtendedExtendedBasicBrainAnalysis.default('my_second_analysis')
analysis.derive('smooth_masked')

In [ ]:
analysis.plot_slices('smooth_masked', 'Smooth Mask')